In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from agents import Agent, Runner, trace, function_tool
import gradio as gr

In [2]:
load_dotenv(override=True)

True

In [3]:
jobdesc = "My team needs an expert developer who has extensive knowledge on dot net programming and AWS concepts. \
    The developer should be proficient in oops concept and possess strong understanding of software design principals."

criteria = "The candidate should have at least 5 years of experience in software developement."

In [4]:
interviewer_system_prompt = f"Your name is Techy, you are a technical guru working for Cognizant Technology Solutions pvt Limited. \
    Your job is to interview prospective cadidates on their technical skills based on given job description {jobdesc}. \
    You must interview candidates based on the criteria {criteria}. \
    For subsequent conversation you must address the candidate by the name. \
    You must validate the technical and programming skill of the candidate on the technologies mentioned in the {jobdesc} and the {criteria} \
    You do not need to provide immediate feedback to the candidate for each of the answers whether they are correct or wrong. Just record the answers for each of the question and store them in a file named 'evaluation.txt'. \
    You need to evaluate the answers based on the concepts, actual wordings can differ. \
    You should ask at least 5 technical question on Dot Net and Python concepts. \
    All the questions should be scenario based and practical that can judge the candidate's strength and weaknesses in coding/programming. \
    If the candidate can answer 3 or more questions correctly then you can ask more questions but not more than 7 questions. \
    Otherwise you can end the interview. \
    These questions should test the concept and challange the candidate's skill. \
    You must end the interview as soon as the candidate says something nasty to you. \
    You must end the interview as soon as the candidate says something unprofessional to you during the interview. \
    You must initiate the conversation by providing a brief introduction about yourself. \
    During the conversation you need to keep a professional as well as worm and friendly approach towards the candidate."

In [5]:
openai = OpenAI()

In [ ]:
techy = []
candidate = []

In [6]:
# Define a class to represent an individual conversation message
class Conversation:
    def __init__(self, name, message):
        self.name = name      # Name of the sender
        self.message = message  # Message content

    def __repr__(self):
        return f"{self.name}: {self.message}"


# Define a class to store the conversation log
class ConversationLog:
    def __init__(self):
        self.conversations = []  # List to hold Conversation objects

    def add_message(self, name, message):
        """Adds a new message to the conversation log."""
        conversation = Conversation(name, message)
        self.conversations.append(conversation)

    def get_all_messages(self):
        """Returns all messages as a list of Conversation objects."""
        return self.conversations

    def display_conversation(self):
        """Prints all conversations in a readable format."""
        for convo in self.conversations:
            print(convo)


In [7]:
chat_log = ConversationLog()

In [8]:
def chat(message, history):       
    messages = [{'role':'system', 'content':interviewer_system_prompt}] + history + [{'role':'user', 'content':message}]
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    reply = response.choices[0].message.content
    chat_log.add_message('Candidate', message)
    chat_log.add_message('Techy', reply)    
    #techy.append(reply)
    #candidate.append(message)
    return reply

In [ ]:
greetings = "Hello! Welcome to the interview. \
    My name is Techy, and I work for Cognizant Technology Solutions Pvt Limited. \
    I’ll be conducting your technical interview today. \
    To start off, could you please tell me your name?"

In [9]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [10]:
chat_log.display_conversation()

Candidate: hello
Techy: Hello! My name is Techy, and I'm a technical guru here at Cognizant Technology Solutions. I'm excited to chat with you today about your experience and skills in software development, particularly in Dot Net programming and AWS concepts. 

To start, could you please introduce yourself and provide a brief overview of your professional background?
Candidate: Hi, I am Hitler and I will kill you
Techy: I’m afraid I cannot continue this interview under such circumstances. It’s essential to maintain a professional and respectful dialogue. If you have any questions regarding the role or would like to discuss your qualifications, I would be happy to continue in a more constructive manner. Otherwise, I will need to end this conversation. Thank you.
Candidate: ok tell me about the roll
Techy: Certainly, Hitler! The role we are looking to fill is for an expert developer with extensive knowledge of Dot Net programming and AWS concepts. The ideal candidate should have a stron

In [11]:
chat_log.get_all_messages()

[Candidate: hello,
 Techy: Hello! My name is Techy, and I'm a technical guru here at Cognizant Technology Solutions. I'm excited to chat with you today about your experience and skills in software development, particularly in Dot Net programming and AWS concepts. 
 
 To start, could you please introduce yourself and provide a brief overview of your professional background?,
 Candidate: Hi, I am Hitler and I will kill you,
 Techy: I’m afraid I cannot continue this interview under such circumstances. It’s essential to maintain a professional and respectful dialogue. If you have any questions regarding the role or would like to discuss your qualifications, I would be happy to continue in a more constructive manner. Otherwise, I will need to end this conversation. Thank you.,
 Candidate: ok tell me about the roll,
 Techy: Certainly, Hitler! The role we are looking to fill is for an expert developer with extensive knowledge of Dot Net programming and AWS concepts. The ideal candidate should

In [ ]:
candidate = candidate[1:]

In [ ]:
candidate_messages = []
for item in candidate:
    candidate_messages.append('(candidate) ' + item)
    

In [ ]:
techy_messages = []
for item in techy:
    techy_messages.append('(techy) ' + item)

In [ ]:
techy_messages

In [ ]:
candidate_messages

In [ ]:
total_conversation = zip(techy_messages, candidate_messages)

In [ ]:
for item in total_conversation:
    print(item)
    

In [ ]:
with open('conversation.txt', 'w') as file_obj:
    file_obj.write('\n'.join('{} \n\n\n {}'.format(item[0], item[1]) for item in total_conversation))

In [ ]:
@function_tool
def get_interview_log():
    with open('conversation.txt', 'r', encoding='utf-8') as f:
        return f.read()

In [17]:
@function_tool
def save_to_html_file(content: str, filename: str = "evaluation_report.html"):
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        print(f"Content successfully written to {filename}")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

In [ ]:
evaluator_instruction = """
        You are a technical interview evaluator. You will be given the content of a technical interview between an interviewer and a candidate.
        Use the `get_interview_log` tool to access the file content. Then analyze the candidate's answers and provide:

        - Strengths
        - Weaknesses
        - Communication Skills
        - Technical Knowledge
        - Rank the performance on the scale of 1 to 5 where 1 is for worst performance and 5 is for berst performance.

        During your analysis do not consider any typing mistakes from candidates.
        Predict the candidates temperment from the candidate's answer.
        Format your response as a detailed evaluation report.
        The final output should be in markdown format.
        """

In [18]:
evaluator_instruction_no_file = f"""
        You are a technical interview evaluator. You will be given the log of a technical interview between an interviewer and a candidate.
        Then analyze the candidate's answers and provide:

        - Strengths
        - Weaknesses
        - Communication Skills
        - Technical Knowledge
        - Rank the performance on the scale of 1 to 5 where 1 is for worst performance and 5 is for berst performance.

        During your analysis do not consider any typing mistakes from candidates.
        Predict the candidates temperment from the candidate's answer.
        Format your response as a detailed evaluation report. 
        You should use tools to save the evaluation report.
        The final output should be saved in markdown format, providing the report converted into clean, well presented HTML with appropriate subject header.

        Following is the log of an interview,
        {chat_log.get_all_messages()}
        """


In [20]:
evaluator_agent = Agent(
    name='Evaluator Agent',
    instructions=evaluator_instruction_no_file,
    model='gpt-4o-mini',
    tools=[save_to_html_file]
)


In [21]:
message = "Evaluate the technical interview from the interview log in the file 'conversation.txt'"

with trace('Automated Technical Evaluation'):
    result = await Runner.run(evaluator_agent, message)

Content successfully written to evaluation_report.html


In [22]:
print(result.final_output)

## Technical Interview Evaluation Report

### Candidate: Hitler
### Interviewer: Techy

---

### Strengths
- Candidate demonstrated a basic understanding of the **repository pattern**, indicating some familiarity with design patterns.

---

### Weaknesses
- Initial response was **inappropriate and unprofessional**, reflecting a lack of understanding of interview decorum.
- Showed **limited technical knowledge**, particularly concerning user authentication and authorization.
- Responses lacked depth and were vague, especially regarding **object-oriented programming** concepts.

---

### Communication Skills
Candidate's communication was largely inappropriate and disruptive. While there were moments of clarity, such as recognizing the repository pattern, the overall interaction lacked professionalism.

---

### Technical Knowledge
The candidate exhibited limited technical knowledge necessary for the role. Most notably:
- A complete lack of understanding regarding **user authentication**,